In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import time
import psutil
import os
import json
import subprocess
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🔧 Ultimate Grid Search 시스템 초기화 + 전원 관리")
print("=" * 60)

# 절전 모드 설정 체크 및 경고
def check_power_settings():
    """시스템의 절전 설정을 체크하고 경고를 표시"""
    
    print("\n⚡ 전원 관리 설정 자동 체크")
    print("-" * 40)
    
    warnings_found = []
    
    try:
        # AC 전원 상태 확인
        try:
            with open('/sys/class/power_supply/ADP1/online', 'r') as f:
                ac_connected = f.read().strip() == '1'
        except:
            result = subprocess.run(['cat', '/sys/class/power_supply/A*/online'], 
                                  capture_output=True, text=True, shell=True)
            ac_connected = '1' in result.stdout
    except:
        ac_connected = False
    
    if ac_connected:
        print("✅ AC 전원 연결됨")
    else:
        print("⚠️  배터리 모드 - 절전 위험 높음!")
        warnings_found.append("배터리 모드")
    
    # GNOME 절전 설정 확인
    try:
        result = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                               'sleep-inactive-ac-timeout'], capture_output=True, text=True)
        ac_timeout = int(result.stdout.strip())
        
        if ac_timeout > 0 and ac_timeout < 7200:  # 2시간 미만
            print(f"⚠️  AC 모드 자동 절전: {ac_timeout//60}분 후")
            warnings_found.append(f"AC 절전 {ac_timeout//60}분")
        else:
            print("✅ AC 모드 절전 설정 양호")
            
    except:
        print("❓ GNOME 절전 설정 확인 불가")
    
    try:
        result = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                               'sleep-inactive-battery-timeout'], capture_output=True, text=True)
        battery_timeout = int(result.stdout.strip())
        
        if battery_timeout > 0 and battery_timeout < 3600:  # 1시간 미만
            print(f"⚠️  배터리 모드 자동 절전: {battery_timeout//60}분 후")
            warnings_found.append(f"배터리 절전 {battery_timeout//60}분")
            
    except:
        print("❓ 배터리 절전 설정 확인 불가")
    
    # 노트북 뚜껑 설정 확인
    try:
        with open('/etc/systemd/logind.conf', 'r') as f:
            content = f.read()
            if '#HandleLidSwitch=suspend' in content:
                print("⚠️  노트북 뚜껑 닫기 → suspend 모드 (기본값)")
                warnings_found.append("뚜껑 닫기 suspend")
            elif 'HandleLidSwitch=ignore' in content:
                print("✅ 노트북 뚜껑 닫기 무시 설정")
    except:
        print("❓ 시스템 뚜껑 설정 확인 불가")
    
    # 경고 요약
    if warnings_found:
        print(f"\n🚨 절전 위험 발견: {len(warnings_found)}개")
        for warning in warnings_found:
            print(f"   • {warning}")
        print(f"\n💡 자동 해결 방법이 아래에 준비되어 있습니다!")
        return False
    else:
        print(f"\n✅ 전원 설정 완전 안전!")
        return True

# 전원 설정 체크 실행
power_safe = check_power_settings()

# 시스템 정보 확인
def get_system_info():
    cpu_count = psutil.cpu_count(logical=True)
    memory_gb = psutil.virtual_memory().total / (1024**3)
    disk_free_gb = psutil.disk_usage('.').free / (1024**3)
    
    print(f"\n🖥️  시스템 리소스 정보:")
    print(f"   CPU 코어: {cpu_count}개")
    print(f"   메모리: {memory_gb:.1f}GB")
    print(f"   디스크 여유공간: {disk_free_gb:.1f}GB")
    
    if disk_free_gb < 2:
        print("⚠️  경고: 디스크 공간이 부족할 수 있습니다!")
    
    return cpu_count, memory_gb, disk_free_gb

cpu_count, memory_gb, disk_free_gb = get_system_info()

# 데이터 로드
print(f"\n📊 데이터 로드 중...")
with open('../../data/processed/df_selected_05.pkl', 'rb') as f:
    df = pickle.load(f)

y = df['SalePrice']
X = df.drop('SalePrice', axis=1)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print(f"   데이터 형태: X {X.shape}, y {y.shape}")

# 기준점 설정
baseline_r2 = 0.8335
random_search_best = 0.8426

print(f"   기준선 R²: {baseline_r2:.4f}")
print(f"   Random Search 최고: {random_search_best:.4f}")
print(f"   목표: Random Search 성능 뛰어넘기")

print(f"\n" + "="*60)
print(f"✅ 시스템 초기화 완료!")


In [ ]:
# 🛡️ 안전 모드 설정 (절전 위험 자동 해결)
APPLY_SAFE_MODE = False  # 절전 위험 발견 시 True로 변경하여 자동 해결

def apply_safe_mode():
    """Grid Search 실행을 위한 안전 모드 설정"""
    
    print("🛡️ 안전 모드 설정 적용 중...")
    
    try:
        # 1. GNOME 절전 설정 임시 비활성화
        print("   1. GNOME 절전 모드 비활성화...")
        
        # 현재 설정 백업
        result_ac = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                                  'sleep-inactive-ac-timeout'], capture_output=True, text=True)
        result_battery = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                                       'sleep-inactive-battery-timeout'], capture_output=True, text=True)
        
        original_ac = result_ac.stdout.strip()
        original_battery = result_battery.stdout.strip()
        
        # 백업 파일 저장
        backup_settings = {
            'ac_timeout': original_ac,
            'battery_timeout': original_battery,
            'timestamp': datetime.now().isoformat()
        }
        
        with open('power_settings_backup.json', 'w') as f:
            json.dump(backup_settings, f, indent=2)
        
        print(f"      백업 저장: AC={original_ac}초, Battery={original_battery}초")
        
        # 절전 모드 비활성화 (0 = 비활성화)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-ac-timeout', '0'], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-battery-timeout', '0'], check=True)
        
        print("      ✅ GNOME 절전 모드 비활성화 완료")
        
        # 2. 화면 보호기 비활성화
        print("   2. 화면 보호기 비활성화...")
        
        try:
            subprocess.run(['gsettings', 'set', 'org.gnome.desktop.screensaver', 
                           'idle-activation-enabled', 'false'], check=True)
            subprocess.run(['gsettings', 'set', 'org.gnome.desktop.session', 
                           'idle-delay', '0'], check=True)
            print("      ✅ 화면 보호기 비활성화 완료")
        except:
            print("      ⚠️ 화면 보호기 설정 실패 (계속 진행)")
        
        print("   3. 안전 모드 적용 완료!")
        print("      ⚠️ Grid Search 완료 후 자동으로 설정이 복원됩니다!")
        
        return True
        
    except Exception as e:
        print(f"   ❌ 안전 모드 설정 실패: {str(e)}")
        print("   → nohup 백그라운드 실행을 사용하세요.")
        return False

def restore_power_settings():
    """백업된 전원 설정을 복원"""
    
    try:
        print("🔄 전원 설정 복원 중...")
        
        with open('power_settings_backup.json', 'r') as f:
            backup = json.load(f)
        
        # 원래 설정 복원
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-ac-timeout', backup['ac_timeout']], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-battery-timeout', backup['battery_timeout']], check=True)
        
        # 화면 보호기 복원
        subprocess.run(['gsettings', 'set', 'org.gnome.desktop.screensaver', 
                       'idle-activation-enabled', 'true'], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.desktop.session', 
                       'idle-delay', '300'], check=True)
        
        print(f"   ✅ 전원 설정 복원 완료")
        print(f"   복원된 설정: AC={backup['ac_timeout']}초, Battery={backup['battery_timeout']}초")
        
        # 백업 파일 삭제
        os.remove('power_settings_backup.json')
        
        return True
        
    except FileNotFoundError:
        print("   ❓ 백업 파일이 없습니다. (설정 변경 없음)")
        return False
    except Exception as e:
        print(f"   ❌ 설정 복원 실패: {str(e)}")
        return False

# 안전 모드 적용 여부 결정
if not power_safe and APPLY_SAFE_MODE:
    print("🚨 전원 위험 감지! 안전 모드 자동 적용 중...")
    safe_mode_applied = apply_safe_mode()
elif not power_safe:
    print("🚨 절전 위험 감지! 해결 방법:")
    print("   ✅ 추천: APPLY_SAFE_MODE = True로 변경 후 재실행 (자동 해결)")
    print("   📱 대안: 아래 nohup 백그라운드 실행 사용")
    safe_mode_applied = False
else:
    print("✅ 전원 설정이 안전합니다. 추가 설정 불필요!")
    safe_mode_applied = False

# 백그라운드 실행 안내 (절전 위험이 있는 경우)
if not power_safe and not safe_mode_applied:
    print(f"\n🖥️ 백그라운드 실행 (nohup) 방법:")
    print(f"   1. 터미널에서 다음 명령어 실행:")
    print(f"      cd {os.getcwd()}")
    print(f"      nohup jupyter nbconvert --execute --to notebook ultimate_grid_search_strategy.ipynb &")
    print(f"   2. 진행 상황 확인:")
    print(f"      tail -f nohup.out")
    print(f"   💡 이 방법으로 노트북 뚜껑을 닫아도 안전하게 실행됩니다!")

print(f"\n" + "="*60)


In [ ]:
print("🎯 Ultimate 파라미터 그리드 정의 및 테스트")
print("=" * 50)

# Random Search 상위 결과 기반으로 설계된 전체 그리드
ultimate_param_grid = {
    'n_estimators': [500, 600, 650, 700, 750, 800],           # 6개
    'max_depth': [None, 10, 25, 35],                          # 4개  
    'min_samples_split': [3, 5, 7, 10, 13],                  # 5개
    'min_samples_leaf': [2, 3, 4, 5, 6],                     # 5개
    'max_features': [0.3, 0.5, 0.7, 'sqrt'],                 # 4개
    'max_samples': [0.65, 0.7, 0.8, 0.9],                    # 4개
    'bootstrap': [True]                                        # 1개 (고정)
}

# 총 조합 수 계산
total_combinations = 1
for param, values in ultimate_param_grid.items():
    total_combinations *= len(values)
    print(f"   {param}: {len(values)}개 값 → {values}")

print(f"\n🔢 총 조합 수: {total_combinations:,}개")
print(f"📊 Random Search 대비: {total_combinations/100:.0f}배 더 많은 탐색")

# 메모리 사용량 추정
estimated_memory_mb = total_combinations * 0.1  # 조합당 약 0.1MB 추정
print(f"💾 예상 메모리 사용량: {estimated_memory_mb:.0f}MB ({estimated_memory_mb/1024:.1f}GB)")

if estimated_memory_mb > memory_gb * 1024 * 0.8:
    print("⚠️  경고: 메모리 사용량이 높을 수 있습니다!")

print(f"\n⏱️  성능 및 시간 테스트 시작...")
print("-" * 40)

# 테스트용 작은 그리드 (12개 조합)
test_grid = {
    'n_estimators': [600, 700, 750],           # 3개
    'max_depth': [None, 25],                   # 2개
    'min_samples_split': [5, 10],              # 2개  
    'min_samples_leaf': [3],                   # 1개 (고정)
    'max_features': [0.7],                     # 1개 (고정)
    'max_samples': [0.8],                      # 1개 (고정)
    'bootstrap': [True]                        # 1개 (고정)
}

test_combinations = 1
for values in test_grid.values():
    test_combinations *= len(values)

print(f"🔬 테스트 그리드: {test_combinations}개 조합")
print(f"📊 전체 대비: {test_combinations/total_combinations*100:.3f}%")

# 단일 조합 시간 측정
print(f"\n1️⃣  단일 조합 시간 측정...")
single_start = time.time()

test_rf = RandomForestRegressor(
    n_estimators=650, max_depth=None, min_samples_split=5,
    min_samples_leaf=3, max_features=0.7, max_samples=0.8,
    bootstrap=True, random_state=42, n_jobs=-1
)
single_scores = cross_val_score(test_rf, X, y, cv=kfold, scoring='r2')

single_end = time.time()
single_time = single_end - single_start

print(f"   단일 조합 소요 시간: {single_time:.2f}초")
print(f"   단일 조합 R² 점수: {single_scores.mean():.6f}")

# 테스트 Grid Search 실행
print(f"\n2️⃣  테스트 Grid Search 실행... ({test_combinations}개 조합)")
test_start = time.time()

rf_test = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_test = GridSearchCV(
    rf_test, test_grid,
    cv=kfold, scoring='r2', n_jobs=1,  # Grid Search는 n_jobs=1 (내부에서 RF가 멀티프로세싱)
    verbose=1, return_train_score=True
)

grid_test.fit(X, y)
test_end = time.time()

test_grid_time = test_end - test_start
time_per_combination = test_grid_time / test_combinations

print(f"\n📊 테스트 결과:")
print(f"   테스트 Grid Search 총 시간: {test_grid_time:.1f}초")
print(f"   조합당 평균 시간: {time_per_combination:.2f}초")
print(f"   최고 R² 점수: {grid_test.best_score_:.6f}")
print(f"   최적 파라미터: {grid_test.best_params_}")

# 전체 실행 시간 예측
estimated_total_time = total_combinations * time_per_combination
estimated_hours = estimated_total_time / 3600

print(f"\n🕐 전체 실행 시간 예측:")
print(f"   예상 총 시간: {estimated_total_time/60:.0f}분 ({estimated_hours:.1f}시간)")
print(f"   예상 완료 시각: {(datetime.now() + timedelta(seconds=estimated_total_time)).strftime('%Y-%m-%d %H:%M')}")

# 경고 및 권장사항
if estimated_hours > 12:
    print(f"\n⚠️  주의사항:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간으로 매우 깁니다")
    print(f"   • 밤사이 실행하거나 주말에 실행하는 것을 권장합니다")
    print(f"   • 시스템이 중단되지 않도록 주의하세요")
elif estimated_hours > 6:
    print(f"\n🟡 권장사항:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간입니다")
    print(f"   • 충분한 시간을 확보한 후 실행하세요")
else:
    print(f"\n✅ 실행 가능:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간으로 적절합니다")

# 성능 개선 가능성 분석
if grid_test.best_score_ > random_search_best:
    improvement = grid_test.best_score_ - random_search_best
    print(f"\n🎯 성능 개선 전망:")
    print(f"   • 테스트에서 Random Search 대비 +{improvement:.6f} R² 개선")
    print(f"   • 전체 탐색에서 더 큰 개선 기대 가능")
else:
    print(f"\n🤔 성능 분석:")
    print(f"   • 테스트에서는 큰 개선이 없었습니다")
    print(f"   • 하지만 전체 탐색에서는 더 나은 결과 가능")

print(f"\n" + "="*50)
print(f"✅ 테스트 완료! 아래에서 실행 여부를 결정하세요.")


In [ ]:
# 🚀 Ultimate Grid Search 실행
EXECUTE_FULL_SEARCH = False  # True로 변경하면 실제 실행

print(f"🚦 Ultimate Grid Search 실행 준비")
print("=" * 50)

# 최종 안전성 체크
print(f"📋 최종 안전성 체크:")
print(f"   • 전원 설정: {'✅ 안전' if power_safe else '⚠️ 위험 (안전모드: ' + str(safe_mode_applied) + ')'}")
print(f"   • 예상 시간: {estimated_hours:.1f}시간")
print(f"   • 시스템 리소스: {cpu_count}코어, {memory_gb:.1f}GB 메모리")
print(f"   • 디스크 공간: {disk_free_gb:.1f}GB 여유")

if not EXECUTE_FULL_SEARCH:
    print(f"\n🛑 Grid Search 실행이 비활성화되어 있습니다.")
    print(f"   실행하려면: EXECUTE_FULL_SEARCH = True로 변경 후 재실행")
    print(f"   ⚠️ 실행 전 위의 테스트 결과와 안전성을 꼼꼼히 검토하세요!")
    
    # 체크리스트 제공
    print(f"\n📝 실행 전 체크리스트:")
    print(f"   □ AC 전원 연결 확인")
    print(f"   □ 예상 {estimated_hours:.1f}시간 동안 시스템 사용 가능")
    print(f"   □ 중요한 작업이 없는 시간대")
    print(f"   □ 안정적인 네트워크 환경")
    if not power_safe and not safe_mode_applied:
        print(f"   □ 절전 위험 해결 (APPLY_SAFE_MODE=True 또는 nohup 사용)")
    
else:
    print(f"\n🚀 Ultimate Grid Search 실행 시작!")
    print("=" * 50)
    
    # 실행 준비
    start_time = datetime.now()
    
    # 파일명 생성
    timestamp = start_time.strftime('%Y%m%d_%H%M%S')
    checkpoint_file = f"ultimate_grid_search_checkpoint_{timestamp}.json"
    results_file = f"ultimate_grid_search_results_{timestamp}.pkl"
    
    print(f"📁 체크포인트 파일: {checkpoint_file}")
    print(f"📁 결과 파일: {results_file}")
    
    # 체크포인트 저장 함수
    def save_checkpoint(current_best_score, current_best_params, combinations_completed):
        checkpoint_data = {
            'timestamp': datetime.now().isoformat(),
            'combinations_completed': combinations_completed,
            'total_combinations': total_combinations,
            'current_best_score': current_best_score,
            'current_best_params': current_best_params,
            'elapsed_time_seconds': (datetime.now() - start_time).total_seconds(),
            'results_file': results_file,
            'power_safe': power_safe,
            'safe_mode_applied': safe_mode_applied
        }
        
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f, indent=2, default=str)
    
    try:
        print(f"🔄 Grid Search 시작...")
        search_start_time = time.time()
        
        # 실제 Grid Search 실행
        rf_ultimate = RandomForestRegressor(random_state=42, n_jobs=-1)
        grid_ultimate = GridSearchCV(
            rf_ultimate, 
            ultimate_param_grid,
            cv=kfold,
            scoring='r2',
            n_jobs=1,  # Grid 레벨에서는 1, RF 레벨에서 멀티프로세싱
            verbose=2,  # 더 자세한 출력
            return_train_score=True,
            error_score='raise'  # 에러 발생 시 중단
        )
        
        # Grid Search 실행
        grid_ultimate.fit(X, y)
        
        search_end_time = time.time()
        total_elapsed = search_end_time - search_start_time
        
        print(f"\n🎉 Ultimate Grid Search 완료!")
        print("=" * 50)
        
        # 최종 결과 저장
        final_results = {
            'best_score': grid_ultimate.best_score_,
            'best_params': grid_ultimate.best_params_,
            'cv_results': pd.DataFrame(grid_ultimate.cv_results_),
            'total_time_seconds': total_elapsed,
            'total_combinations': total_combinations,
            'completion_time': datetime.now().isoformat(),
            'system_info': {
                'cpu_count': cpu_count,
                'memory_gb': memory_gb,
                'power_safe': power_safe,
                'safe_mode_applied': safe_mode_applied
            }
        }
        
        # 결과 파일 저장
        with open(results_file, 'wb') as f:
            pickle.dump(final_results, f)
        
        # 최종 체크포인트 저장
        save_checkpoint(grid_ultimate.best_score_, grid_ultimate.best_params_, total_combinations)
        
        print(f"📊 최종 결과:")
        print(f"   최고 R² 점수: {grid_ultimate.best_score_:.6f}")
        print(f"   Random Search 대비 향상: {grid_ultimate.best_score_ - random_search_best:+.6f}")
        print(f"   기준선 대비 향상: {grid_ultimate.best_score_ - baseline_r2:+.6f}")
        print(f"   총 소요 시간: {total_elapsed/3600:.1f}시간")
        print(f"   결과 파일: {results_file}")
        
        # 최적 파라미터 출력
        print(f"\n🏆 최적 하이퍼파라미터:")
        for param, value in grid_ultimate.best_params_.items():
            print(f"   {param}: {value}")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"\n🔄 안전 모드 설정 자동 복원 중...")
            restore_success = restore_power_settings()
            if restore_success:
                print(f"✅ 전원 설정 복원 완료!")
            else:
                print(f"⚠️ 자동 복원 실패 - 수동 복원 필요")
        
        print(f"\n🎊 Ultimate Grid Search 성공적으로 완료!")
            
    except KeyboardInterrupt:
        print(f"\n🛑 사용자에 의해 중단되었습니다.")
        print(f"체크포인트 파일 '{checkpoint_file}'를 확인하여 진행 상황을 파악할 수 있습니다.")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"🔄 중단 시 안전 모드 설정 복원 중...")
            restore_power_settings()
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        print(f"체크포인트 파일 '{checkpoint_file}'를 확인하여 진행 상황을 파악할 수 있습니다.")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"🔄 오류 시 안전 모드 설정 복원 중...")
            restore_power_settings()

# 실행 요약
print(f"\n💡 참고 정보:")
print(f"   • 전체 조합 수: {total_combinations:,}개")
print(f"   • 예상 소요 시간: {estimated_hours:.1f}시간")
print(f"   • 예상 메모리 사용량: {estimated_memory_mb/1024:.1f}GB")
print(f"   • CPU 코어 활용: {cpu_count}개 코어")

if EXECUTE_FULL_SEARCH:
    print(f"   • 실행 상태: ✅ 활성화 - Grid Search 실행됨")
else:
    print(f"   • 실행 상태: 🛑 비활성화 (EXECUTE_FULL_SEARCH = False)")
    print(f"   • 실행하려면: EXECUTE_FULL_SEARCH = True로 변경 후 재실행")

print(f"\n" + "="*50)


In [ ]:
# 🔄 수동 설정 복원 (Grid Search 완료 후 필요시 실행)
RESTORE_SETTINGS = False  # True로 변경하면 설정 복원

print("🔧 전원 설정 수동 복원")
print("=" * 30)

if RESTORE_SETTINGS:
    print("🔄 전원 설정 수동 복원 실행...")
    success = restore_power_settings()
    
    if success:
        print("✅ 설정 복원 완료!")
        print("이제 시스템이 정상적인 절전 모드로 돌아갑니다.")
    else:
        print("❌ 자동 복원 실패. 수동 복원 방법:")
        print("   1. 시스템 설정 → 전원 관리")
        print("   2. '절전 모드' 설정을 원하는 값으로 변경")
        print("   3. AC: 보통 1시간(3600초), 배터리: 15분(900초)")
        
        print("\n또는 터미널에서:")
        print("   gsettings set org.gnome.settings-daemon.plugins.power sleep-inactive-ac-timeout 3600")
        print("   gsettings set org.gnome.settings-daemon.plugins.power sleep-inactive-battery-timeout 900")
        
else:
    print("🔄 설정 복원이 비활성화되어 있습니다.")
    print("   복원하려면: RESTORE_SETTINGS = True로 변경 후 실행")
    
    # 현재 백업 파일 상태 확인
    if os.path.exists('power_settings_backup.json'):
        try:
            with open('power_settings_backup.json', 'r') as f:
                backup = json.load(f)
            print(f"\n📋 백업된 설정 발견:")
            print(f"   AC 절전: {backup['ac_timeout']}초")
            print(f"   배터리 절전: {backup['battery_timeout']}초")
            print(f"   백업 시간: {backup['timestamp']}")
            print(f"   ✅ 복원 준비 완료!")
            print(f"\n💡 Grid Search 완료 후 RESTORE_SETTINGS = True로 설정하여 복원하세요.")
        except:
            print("\n❓ 백업 파일이 손상되었을 수 있습니다.")
    else:
        print(f"\n📋 백업 파일 없음")
        print(f"   • 안전 모드를 사용하지 않았거나")
        print(f"   • 이미 설정이 복원되었을 수 있습니다.")

print(f"\n" + "="*30)


In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import time
import psutil
import os
import json
import subprocess
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🔧 Ultimate Grid Search 시스템 초기화")
print("=" * 50)

# 절전 모드 설정 체크 및 경고
def check_power_settings():
    """시스템의 절전 설정을 체크하고 경고를 표시"""
    
    print("\n⚡ 전원 관리 설정 체크")
    print("-" * 30)
    
    warnings_found = []
    
    try:
        # AC 전원 상태 확인
        with open('/sys/class/power_supply/ADP1/online', 'r') as f:
            ac_connected = f.read().strip() == '1'
    except:
        try:
            result = subprocess.run(['cat', '/sys/class/power_supply/A*/online'], 
                                  capture_output=True, text=True, shell=True)
            ac_connected = '1' in result.stdout
        except:
            ac_connected = False
    
    if ac_connected:
        print("✅ AC 전원 연결됨")
    else:
        print("⚠️  배터리 모드 - 절전 위험 높음!")
        warnings_found.append("배터리 모드")
    
    # GNOME 절전 설정 확인
    try:
        result = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                               'sleep-inactive-ac-timeout'], capture_output=True, text=True)
        ac_timeout = int(result.stdout.strip())
        
        if ac_timeout > 0 and ac_timeout < 7200:  # 2시간 미만
            print(f"⚠️  AC 모드 자동 절전: {ac_timeout//60}분 후")
            warnings_found.append(f"AC 절전 {ac_timeout//60}분")
        else:
            print("✅ AC 모드 절전 설정 양호")
            
    except:
        print("❓ GNOME 절전 설정 확인 불가")
    
    try:
        result = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                               'sleep-inactive-battery-timeout'], capture_output=True, text=True)
        battery_timeout = int(result.stdout.strip())
        
        if battery_timeout > 0 and battery_timeout < 3600:  # 1시간 미만
            print(f"⚠️  배터리 모드 자동 절전: {battery_timeout//60}분 후")
            warnings_found.append(f"배터리 절전 {battery_timeout//60}분")
            
    except:
        print("❓ 배터리 절전 설정 확인 불가")
    
    # 노트북 뚜껑 설정 확인
    try:
        with open('/etc/systemd/logind.conf', 'r') as f:
            content = f.read()
            if '#HandleLidSwitch=suspend' in content:
                print("⚠️  노트북 뚜껑 닫기 → suspend 모드 (기본값)")
                warnings_found.append("뚜껑 닫기 suspend")
            elif 'HandleLidSwitch=ignore' in content:
                print("✅ 노트북 뚜껑 닫기 무시 설정")
            else:
                print("❓ 노트북 뚜껑 설정 확인 불가")
    except:
        print("❓ 시스템 뚜껑 설정 확인 불가")
    
    # 경고 요약
    if warnings_found:
        print(f"\n🚨 주의사항 발견: {len(warnings_found)}개")
        for warning in warnings_found:
            print(f"   • {warning}")
        
        print(f"\n💡 권장 해결책:")
        print(f"   1. AC 전원을 연결하고 유지하세요")
        print(f"   2. 노트북을 열어둔 상태로 실행하세요")
        print(f"   3. 아래의 '안전 모드' 설정을 적용하세요")
        print(f"   4. nohup이나 screen을 사용하여 백그라운드 실행 고려")
        
        return False
    else:
        print(f"\n✅ 전원 설정 안전함")
        return True

# 전원 설정 체크 실행
power_safe = check_power_settings()

# 시스템 정보 확인
def get_system_info():
    cpu_count = psutil.cpu_count(logical=True)
    memory_gb = psutil.virtual_memory().total / (1024**3)
    disk_free_gb = psutil.disk_usage('.').free / (1024**3)
    
    print(f"🖥️  시스템 정보:")
    print(f"   CPU 코어: {cpu_count}개")
    print(f"   메모리: {memory_gb:.1f}GB")
    print(f"   디스크 여유공간: {disk_free_gb:.1f}GB")
    
    if disk_free_gb < 2:
        print("⚠️  경고: 디스크 공간이 부족할 수 있습니다!")
    
    return cpu_count, memory_gb, disk_free_gb

cpu_count, memory_gb, disk_free_gb = get_system_info()

# 데이터 로드
print(f"\n📊 데이터 로드 중...")
with open('../../data/processed/df_selected_05.pkl', 'rb') as f:
    df = pickle.load(f)

y = df['SalePrice']
X = df.drop('SalePrice', axis=1)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print(f"   데이터 형태: X {X.shape}, y {y.shape}")

# 기준점 설정
baseline_r2 = 0.8335
random_search_best = 0.8426

print(f"   기준선 R²: {baseline_r2:.4f}")
print(f"   Random Search 최고: {random_search_best:.4f}")
print(f"   목표: Random Search 성능 뛰어넘기")


🔧 Ultimate Grid Search 시스템 초기화
🖥️  시스템 정보:
   CPU 코어: 16개
   메모리: 14.9GB
   디스크 여유공간: 26.7GB

📊 데이터 로드 중...
   데이터 형태: X (1460, 15), y (1460,)
   기준선 R²: 0.8335
   Random Search 최고: 0.8426
   목표: Random Search 성능 뛰어넘기


In [ ]:
print("\n🎯 전체 파라미터 그리드 정의")
print("=" * 40)

# Random Search 상위 결과 기반으로 설계된 전체 그리드
# Random Search 상위 결과를 반영한 개선된 파라미터 그리드
ultimate_param_grid = {
    'n_estimators': [500, 600, 646, 650, 700, 750, 800],     # 7개 (646 추가!)
    'max_depth': [None, 10, 25, 35],                          # 4개  
    'min_samples_split': [3, 5, 7, 10, 13],                  # 5개
    'min_samples_leaf': [2, 3, 4, 5, 6],                     # 5개
    'max_features': [0.3, 0.5, 0.7, 'sqrt'],                 # 4개
    'max_samples': [0.65, 0.69, 0.7, 0.8, 0.9],             # 5개 (0.69 추가!)
    'bootstrap': [True]                                        # 1개 (고정)
}

# 총 조합 수 계산
total_combinations = 1
for param, values in ultimate_param_grid.items():
    total_combinations *= len(values)
    print(f"   {param}: {len(values)}개 값 → {values}")

print(f"\n🔢 총 조합 수: {total_combinations:,}개")
print(f"📊 Random Search 대비: {total_combinations/100:.0f}배 더 많은 탐색")

# 메모리 사용량 추정
estimated_memory_mb = total_combinations * 0.1  # 조합당 약 0.1MB 추정
print(f"💾 예상 메모리 사용량: {estimated_memory_mb:.0f}MB ({estimated_memory_mb/1024:.1f}GB)")

if estimated_memory_mb > memory_gb * 1024 * 0.8:
    print("⚠️  경고: 메모리 사용량이 높을 수 있습니다!")



🎯 전체 파라미터 그리드 정의
   n_estimators: 6개 값 → [500, 600, 650, 700, 750, 800]
   max_depth: 4개 값 → [None, 10, 25, 35]
   min_samples_split: 5개 값 → [3, 5, 7, 10, 13]
   min_samples_leaf: 5개 값 → [2, 3, 4, 5, 6]
   max_features: 4개 값 → [0.3, 0.5, 0.7, 'sqrt']
   max_samples: 4개 값 → [0.65, 0.7, 0.8, 0.9]
   bootstrap: 1개 값 → [True]

🔢 총 조합 수: 9,600개
📊 Random Search 대비: 96배 더 많은 탐색
💾 예상 메모리 사용량: 960MB (0.9GB)


In [ ]:
print("\n⏱️  성능 및 시간 테스트 실행")
print("=" * 40)

# 테스트용 작은 그리드 (15개 조합)
test_grid = {
    'n_estimators': [600, 700, 750],           # 3개
    'max_depth': [None, 25],                   # 2개
    'min_samples_split': [5, 10],              # 2개  
    'min_samples_leaf': [3],                   # 1개 (고정)
    'max_features': [0.7],                     # 1개 (고정)
    'max_samples': [0.8],                      # 1개 (고정)
    'bootstrap': [True]                        # 1개 (고정)
}

test_combinations = 1
for values in test_grid.values():
    test_combinations *= len(values)

print(f"🔬 테스트 그리드: {test_combinations}개 조합")
print(f"📊 전체 대비: {test_combinations/total_combinations*100:.3f}%")

# 단일 조합 시간 측정
print(f"\n1️⃣  단일 조합 시간 측정...")
single_start = time.time()

test_rf = RandomForestRegressor(
    n_estimators=650, max_depth=None, min_samples_split=5,
    min_samples_leaf=3, max_features=0.7, max_samples=0.8,
    bootstrap=True, random_state=42, n_jobs=-1
)
single_scores = cross_val_score(test_rf, X, y, cv=kfold, scoring='r2')

single_end = time.time()
single_time = single_end - single_start

print(f"   단일 조합 소요 시간: {single_time:.2f}초")
print(f"   단일 조합 R² 점수: {single_scores.mean():.6f}")

# 테스트 Grid Search 실행
print(f"\n2️⃣  테스트 Grid Search 실행... ({test_combinations}개 조합)")
test_start = time.time()

rf_test = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_test = GridSearchCV(
    rf_test, test_grid,
    cv=kfold, scoring='r2', n_jobs=1,  # Grid Search는 n_jobs=1 (내부에서 RF가 멀티프로세싱)
    verbose=1, return_train_score=True
)

grid_test.fit(X, y)
test_end = time.time()

test_grid_time = test_end - test_start
time_per_combination = test_grid_time / test_combinations

print(f"\n📊 테스트 결과:")
print(f"   테스트 Grid Search 총 시간: {test_grid_time:.1f}초")
print(f"   조합당 평균 시간: {time_per_combination:.2f}초")
print(f"   최고 R² 점수: {grid_test.best_score_:.6f}")
print(f"   최적 파라미터: {grid_test.best_params_}")

# 전체 실행 시간 예측
estimated_total_time = total_combinations * time_per_combination
estimated_hours = estimated_total_time / 3600

print(f"\n🕐 전체 실행 시간 예측:")
print(f"   예상 총 시간: {estimated_total_time/60:.1f}분 ({estimated_hours:.1f}시간)")
print(f"   예상 완료 시각: {(datetime.now() + timedelta(seconds=estimated_total_time)).strftime('%Y-%m-%d %H:%M')}")

# 경고 및 권장사항
if estimated_hours > 12:
    print(f"\n⚠️  주의사항:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간으로 매우 깁니다")
    print(f"   • 밤사이 실행하거나 주말에 실행하는 것을 권장합니다")
    print(f"   • 시스템이 중단되지 않도록 주의하세요")
elif estimated_hours > 6:
    print(f"\n🟡 권장사항:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간입니다")
    print(f"   • 충분한 시간을 확보한 후 실행하세요")
else:
    print(f"\n✅ 실행 가능:")
    print(f"   • 예상 시간이 {estimated_hours:.1f}시간으로 적절합니다")

# 성능 개선 가능성 분석
if grid_test.best_score_ > random_search_best:
    improvement = grid_test.best_score_ - random_search_best
    print(f"\n🎯 성능 개선 전망:")
    print(f"   • 테스트에서 Random Search 대비 +{improvement:.6f} R² 개선")
    print(f"   • 전체 탐색에서 더 큰 개선 기대 가능")
else:
    print(f"\n🤔 성능 분석:")
    print(f"   • 테스트에서는 큰 개선이 없었습니다")
    print(f"   • 하지만 전체 탐색에서는 더 나은 결과 가능")

print(f"\n" + "="*50)
print(f"✅ 테스트 완료! 위 정보를 확인 후 전체 실행 여부를 결정하세요.")



⏱️  성능 및 시간 테스트 실행
🔬 테스트 그리드: 12개 조합
📊 전체 대비: 0.125%

1️⃣  단일 조합 시간 측정...
   단일 조합 소요 시간: 3.75초
   단일 조합 R² 점수: 0.842791

2️⃣  테스트 Grid Search 실행... (12개 조합)
Fitting 5 folds for each of 12 candidates, totalling 60 fits

📊 테스트 결과:
   테스트 Grid Search 총 시간: 57.9초
   조합당 평균 시간: 4.82초
   최고 R² 점수: 0.842958
   최적 파라미터: {'bootstrap': True, 'max_depth': None, 'max_features': 0.7, 'max_samples': 0.8, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 600}

🕐 전체 실행 시간 예측:
   예상 총 시간: 771.9분 (12.9시간)
   예상 완료 시각: 2025-07-22 11:45

⚠️  주의사항:
   • 예상 시간이 12.9시간으로 매우 깁니다
   • 밤사이 실행하거나 주말에 실행하는 것을 권장합니다
   • 시스템이 중단되지 않도록 주의하세요

🎯 성능 개선 전망:
   • 테스트에서 Random Search 대비 +0.000358 R² 개선
   • 전체 탐색에서 더 큰 개선 기대 가능

✅ 테스트 완료! 위 정보를 확인 후 전체 실행 여부를 결정하세요.


In [ ]:
# 실행 여부 결정 (사용자가 수동으로 설정)
EXECUTE_FULL_SEARCH = False  # True로 변경하면 실제 실행

if not EXECUTE_FULL_SEARCH:
    print("🛑 전체 Grid Search 실행이 비활성화되어 있습니다.")
    print("   실행하려면 EXECUTE_FULL_SEARCH = True로 변경하세요.")
    print("   하지만 먼저 위의 테스트 결과를 신중히 검토하세요!")
    print(f"   예상 소요 시간: {estimated_hours:.1f}시간")
else:
    print("🚀 전체 Grid Search 실행 시작!")
    print("="*50)
    
    # 실행 준비
    start_time = datetime.now()
    
    # 체크포인트 파일명
    checkpoint_file = f"ultimate_grid_search_checkpoint_{start_time.strftime('%Y%m%d_%H%M%S')}.json"
    results_file = f"ultimate_grid_search_results_{start_time.strftime('%Y%m%d_%H%M%S')}.pkl"
    
    print(f"📁 체크포인트 파일: {checkpoint_file}")
    print(f"📁 결과 파일: {results_file}")
    
    # 진행 상황 추적 함수
    def save_checkpoint(current_results, current_best_score, current_best_params, 
                       combinations_completed, start_time):
        checkpoint_data = {
            'timestamp': datetime.now().isoformat(),
            'combinations_completed': combinations_completed,
            'total_combinations': total_combinations,
            'current_best_score': current_best_score,
            'current_best_params': current_best_params,
            'elapsed_time_seconds': (datetime.now() - start_time).total_seconds(),
            'results_file': results_file
        }
        
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f, indent=2, default=str)
    
    # 진행 상황 출력 함수
    def print_progress(combinations_completed, current_best_score, elapsed_time):
        progress_pct = combinations_completed / total_combinations * 100
        estimated_remaining = elapsed_time / combinations_completed * (total_combinations - combinations_completed)
        estimated_completion = datetime.now() + timedelta(seconds=estimated_remaining)
        
        print(f"📊 진행률: {combinations_completed:,}/{total_combinations:,} ({progress_pct:.1f}%)")
        print(f"🏆 현재 최고 점수: {current_best_score:.6f}")
        print(f"⏰ 경과 시간: {elapsed_time/3600:.1f}시간")
        print(f"🕐 예상 완료: {estimated_completion.strftime('%Y-%m-%d %H:%M')}")
        print("-" * 40)
    
    try:
        print("🔄 Grid Search 시작...")
        search_start_time = time.time()
        
        # 실제 Grid Search 실행
        rf_ultimate = RandomForestRegressor(random_state=42, n_jobs=-1)
        grid_ultimate = GridSearchCV(
            rf_ultimate, 
            ultimate_param_grid,
            cv=kfold,
            scoring='r2',
            n_jobs=1,  # Grid 레벨에서는 1, RF 레벨에서 멀티프로세싱
            verbose=2,  # 더 자세한 출력
            return_train_score=True,
            error_score='raise'  # 에러 발생 시 중단
        )
        
        # 커스텀 진행 상황 모니터링을 위한 Grid Search
        # (실제로는 sklearn의 GridSearchCV가 내부적으로 처리)
        grid_ultimate.fit(X, y)
        
        search_end_time = time.time()
        total_elapsed = search_end_time - search_start_time
        
        print("\n🎉 전체 Grid Search 완료!")
        print("="*50)
        
        # 최종 결과 저장
        final_results = {
            'best_score': grid_ultimate.best_score_,
            'best_params': grid_ultimate.best_params_,
            'cv_results': pd.DataFrame(grid_ultimate.cv_results_),
            'total_time_seconds': total_elapsed,
            'total_combinations': total_combinations,
            'completion_time': datetime.now().isoformat()
        }
        
        # 결과 파일 저장
        with open(results_file, 'wb') as f:
            pickle.dump(final_results, f)
        
        # 최종 체크포인트 저장
        save_checkpoint(final_results, grid_ultimate.best_score_, 
                       grid_ultimate.best_params_, total_combinations, start_time)
        
        print(f"📊 최종 결과:")
        print(f"   최고 R² 점수: {grid_ultimate.best_score_:.6f}")
        print(f"   Random Search 대비 향상: {grid_ultimate.best_score_ - random_search_best:+.6f}")
        print(f"   총 소요 시간: {total_elapsed/3600:.1f}시간")
        print(f"   결과 파일: {results_file}")
        
        # 최적 파라미터 출력
        print(f"\n🏆 최적 하이퍼파라미터:")
        for param, value in grid_ultimate.best_params_.items():
            print(f"   {param}: {value}")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"\n🔄 안전 모드 설정 복원 중...")
            restore_power_settings()
            
    except KeyboardInterrupt:
        print("\n🛑 사용자에 의해 중단되었습니다.")
        print("체크포인트 파일을 확인하여 진행 상황을 파악할 수 있습니다.")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"🔄 중단 시 안전 모드 설정 복원 중...")
            restore_power_settings()
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        print("체크포인트 파일을 확인하여 진행 상황을 파악할 수 있습니다.")
        
        # 안전 모드가 적용된 경우 설정 복원
        if safe_mode_applied:
            print(f"🔄 오류 시 안전 모드 설정 복원 중...")
            restore_power_settings()

# 실행 여부와 관계없이 보여줄 정보
print(f"\n💡 참고 정보:")
print(f"   • 전체 조합 수: {total_combinations:,}개")
print(f"   • 예상 소요 시간: {estimated_hours:.1f}시간")
print(f"   • 예상 메모리 사용량: {estimated_memory_mb/1024:.1f}GB")
print(f"   • CPU 코어 활용: {cpu_count}개 코어")

if EXECUTE_FULL_SEARCH:
    print(f"   • 실행 상태: ✅ 활성화")
else:
    print(f"   • 실행 상태: 🛑 비활성화 (EXECUTE_FULL_SEARCH = False)")


In [ ]:
# 결과 분석 (Grid Search가 실행된 경우)
def analyze_results(results_file_path):
    """저장된 결과 파일을 분석하는 함수"""
    
    try:
        print("📊 결과 분석 시작...")
        
        # 결과 파일 로드
        with open(results_file_path, 'rb') as f:
            results = pickle.load(f)
        
        best_score = results['best_score']
        best_params = results['best_params']
        cv_results_df = results['cv_results']
        total_time = results['total_time_seconds']
        
        print(f"\n🏆 Ultimate Grid Search 최종 결과")
        print("="*50)
        print(f"최고 R² 점수: {best_score:.6f}")
        print(f"Random Search 대비 향상: {best_score - random_search_best:+.6f}")
        print(f"기준선 대비 향상: {best_score - baseline_r2:+.6f}")
        print(f"총 소요 시간: {total_time/3600:.1f}시간")
        
        print(f"\n🎯 최적 하이퍼파라미터:")
        for param, value in best_params.items():
            print(f"   {param}: {value}")
        
        # 상위 10개 결과 분석
        top_10 = cv_results_df.nlargest(10, 'mean_test_score')
        print(f"\n🥇 상위 10개 결과:")
        print(top_10[['mean_test_score', 'std_test_score', 'rank_test_score']].round(6))
        
        # 파라미터별 성능 분석
        print(f"\n📈 파라미터별 성능 분석:")
        
        for param in ultimate_param_grid.keys():
            if param == 'bootstrap':
                continue
                
            param_col = f'param_{param}'
            if param_col in cv_results_df.columns:
                param_analysis = cv_results_df.groupby(param_col)['mean_test_score'].agg(['mean', 'std', 'count'])
                print(f"\n{param}:")
                print(param_analysis.round(6))
        
        # 성능 분포 분석
        scores = cv_results_df['mean_test_score']
        print(f"\n📊 성능 분포:")
        print(f"   평균: {scores.mean():.6f}")
        print(f"   표준편차: {scores.std():.6f}")
        print(f"   최고점: {scores.max():.6f}")
        print(f"   최저점: {scores.min():.6f}")
        print(f"   상위 1%: {scores.quantile(0.99):.6f}")
        print(f"   상위 10%: {scores.quantile(0.9):.6f}")
        
        # ROI 분석
        print(f"\n💰 투자 대비 효과 (ROI):")
        time_investment_hours = total_time / 3600
        performance_gain = best_score - random_search_best
        roi_per_hour = performance_gain / time_investment_hours if time_investment_hours > 0 else 0
        
        print(f"   시간 투자: {time_investment_hours:.1f}시간")
        print(f"   성능 향상: {performance_gain:+.6f} R²")
        print(f"   시간당 ROI: {roi_per_hour:.8f} R²/시간")
        
        if performance_gain > 0.005:
            print("   → 🟢 매우 가치 있는 투자")
        elif performance_gain > 0.002:
            print("   → 🟡 가치 있는 투자")
        elif performance_gain > 0:
            print("   → 🔵 약간의 개선")
        else:
            print("   → 🔴 개선 없음")
        
        return results
        
    except FileNotFoundError:
        print(f"❌ 결과 파일을 찾을 수 없습니다: {results_file_path}")
        print("   Grid Search가 아직 실행되지 않았거나 파일명이 다를 수 있습니다.")
        return None
    except Exception as e:
        print(f"❌ 결과 분석 중 오류 발생: {str(e)}")
        return None

# 시각화 함수
def visualize_results(results):
    """결과를 시각화하는 함수"""
    
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        cv_results_df = results['cv_results']
        
        # 그래프 설정
        plt.style.use('default')
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('🚀 Ultimate Grid Search 결과 분석', fontsize=16, fontweight='bold')
        
        # 1. 성능 분포 히스토그램
        axes[0,0].hist(cv_results_df['mean_test_score'], bins=50, alpha=0.7, color='skyblue')
        axes[0,0].axvline(results['best_score'], color='red', linestyle='--', label=f'최고점: {results["best_score"]:.6f}')
        axes[0,0].axvline(random_search_best, color='orange', linestyle='--', label=f'Random Search: {random_search_best:.6f}')
        axes[0,0].set_xlabel('R² Score')
        axes[0,0].set_ylabel('빈도')
        axes[0,0].set_title('성능 분포')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # 2. n_estimators별 성능
        if 'param_n_estimators' in cv_results_df.columns:
            n_est_perf = cv_results_df.groupby('param_n_estimators')['mean_test_score'].mean()
            axes[0,1].bar(range(len(n_est_perf)), n_est_perf.values, color='lightgreen')
            axes[0,1].set_xticks(range(len(n_est_perf)))
            axes[0,1].set_xticklabels(n_est_perf.index)
            axes[0,1].set_xlabel('n_estimators')
            axes[0,1].set_ylabel('평균 R² Score')
            axes[0,1].set_title('n_estimators별 성능')
            axes[0,1].grid(True, alpha=0.3)
        
        # 3. max_depth별 성능
        if 'param_max_depth' in cv_results_df.columns:
            depth_perf = cv_results_df.groupby('param_max_depth')['mean_test_score'].mean()
            axes[0,2].bar(range(len(depth_perf)), depth_perf.values, color='lightcoral')
            axes[0,2].set_xticks(range(len(depth_perf)))
            axes[0,2].set_xticklabels([str(x) for x in depth_perf.index])
            axes[0,2].set_xlabel('max_depth')
            axes[0,2].set_ylabel('평균 R² Score')
            axes[0,2].set_title('max_depth별 성능')
            axes[0,2].grid(True, alpha=0.3)
        
        # 4. 상위 20개 결과
        top_20 = cv_results_df.nlargest(20, 'mean_test_score')
        axes[1,0].plot(range(len(top_20)), top_20['mean_test_score'].values, 'o-', color='purple')
        axes[1,0].set_xlabel('순위')
        axes[1,0].set_ylabel('R² Score')
        axes[1,0].set_title('상위 20개 결과')
        axes[1,0].grid(True, alpha=0.3)
        
        # 5. 성능 vs 표준편차
        axes[1,1].scatter(cv_results_df['mean_test_score'], cv_results_df['std_test_score'], 
                         alpha=0.6, color='orange')
        axes[1,1].set_xlabel('평균 R² Score')
        axes[1,1].set_ylabel('표준편차')
        axes[1,1].set_title('성능 vs 안정성')
        axes[1,1].grid(True, alpha=0.3)
        
        # 6. 파라미터 중요도 (상위 결과 기준)
        top_params = top_20.iloc[0]
        param_values = []
        param_names = []
        
        for param in ultimate_param_grid.keys():
            if param != 'bootstrap':
                param_col = f'param_{param}'
                if param_col in top_params:
                    param_names.append(param.replace('_', '\n'))
                    # 값을 숫자로 변환 (가능한 경우)
                    val = top_params[param_col]
                    if isinstance(val, (int, float)):
                        param_values.append(val)
                    else:
                        param_values.append(hash(str(val)) % 100)  # 문자열의 경우 해시값 사용
        
        if param_values:
            axes[1,2].bar(range(len(param_values)), param_values, color='gold')
            axes[1,2].set_xticks(range(len(param_names)))
            axes[1,2].set_xticklabels(param_names, rotation=45, ha='right')
            axes[1,2].set_ylabel('파라미터 값')
            axes[1,2].set_title('최고 성능 파라미터')
            axes[1,2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # 파일 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'ultimate_grid_search_analysis_{timestamp}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"📊 분석 차트가 '{filename}'로 저장되었습니다.")
        
    except ImportError:
        print("📊 matplotlib이나 seaborn이 설치되지 않아 시각화를 건너뜁니다.")
        print("설치하려면: pip install matplotlib seaborn")
    except Exception as e:
        print(f"📊 시각화 중 오류 발생: {str(e)}")

# 결과 분석 실행 (결과 파일이 있는 경우)
print(f"\n📋 결과 분석 준비:")
print(f"   Grid Search 실행 후 결과 파일이 생성되면 자동으로 분석됩니다.")

# 기존 결과 파일 찾기
import glob
result_files = glob.glob("ultimate_grid_search_results_*.pkl")

if result_files:
    latest_file = max(result_files, key=os.path.getctime)
    print(f"   발견된 결과 파일: {latest_file}")
    
    # 결과 분석 실행
    results = analyze_results(latest_file)
    
    if results:
        # 시각화 실행
        visualize_results(results)
        
        # 최종 모델 저장
        final_model = RandomForestRegressor(**results['best_params'], random_state=42)
        final_model.fit(X, y)
        
        model_filename = f"ultimate_optimized_rf_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
        with open(model_filename, 'wb') as f:
            pickle.dump(final_model, f)
        
        print(f"\n💾 최종 최적화 모델이 '{model_filename}'로 저장되었습니다.")
        
else:
    print(f"   아직 실행된 결과 파일이 없습니다.")
    print(f"   Grid Search 실행 후 결과를 확인하세요.")


In [ ]:
# 최종 실행 요약
print("🎯 Ultimate Grid Search Strategy 준비 완료!")
print("="*60)

print(f"\n📊 설정 요약:")
print(f"   • 총 조합 수: {total_combinations:,}개")
print(f"   • 예상 소요 시간: {estimated_hours:.1f}시간")
print(f"   • CPU 활용: {cpu_count}개 코어")
print(f"   • 메모리 예상 사용량: {estimated_memory_mb/1024:.1f}GB")

print(f"\n🚦 현재 상태:")
if EXECUTE_FULL_SEARCH:
    print(f"   • ✅ 실행 활성화 - Grid Search가 실행됩니다!")
else:
    print(f"   • 🛑 실행 비활성화 - 안전 모드")
    print(f"   • 실행하려면: EXECUTE_FULL_SEARCH = True로 변경")

print(f"\n📋 다음 단계:")
if not EXECUTE_FULL_SEARCH:
    print(f"   1. 위의 테스트 결과를 검토하세요")
    print(f"   2. 시간과 리소스를 확보하세요")
    print(f"   3. EXECUTE_FULL_SEARCH = True로 변경하세요")
    print(f"   4. 해당 셀을 다시 실행하세요")
else:
    print(f"   1. Grid Search 실행 중...")
    print(f"   2. 결과를 기다리세요 (약 {estimated_hours:.1f}시간)")
    print(f"   3. 완료 후 자동으로 결과 분석됩니다")

print(f"\n💡 참고:")
print(f"   • 체크포인트 파일로 진행 상황 추적 가능")
print(f"   • Ctrl+C로 언제든 중단 가능")
print(f"   • 결과는 자동으로 저장됩니다")

print(f"\n🎉 Random Forest 하이퍼파라미터 튜닝의 궁극 전략 준비 완료!")

# 현재 시각 표시
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"\n⏰ 현재 시각: {current_time}")
if not EXECUTE_FULL_SEARCH:
    print(f"   실행 시 예상 완료 시각: {(datetime.now() + timedelta(hours=estimated_hours)).strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n" + "="*60)
print(f"🚀 준비가 되면 EXECUTE_FULL_SEARCH = True로 설정하고 시작하세요!")


In [ ]:
# 안전 모드 설정 (절전 모드 일시 비활성화)
APPLY_SAFE_MODE = False  # True로 변경하면 안전 설정 적용

def apply_safe_mode():
    """Grid Search 실행을 위한 안전 모드 설정"""
    
    print("🛡️ 안전 모드 설정 적용 중...")
    
    try:
        # 1. GNOME 절전 설정 임시 비활성화
        print("   1. GNOME 절전 모드 비활성화...")
        
        # 현재 설정 백업
        result_ac = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                                  'sleep-inactive-ac-timeout'], capture_output=True, text=True)
        result_battery = subprocess.run(['gsettings', 'get', 'org.gnome.settings-daemon.plugins.power', 
                                       'sleep-inactive-battery-timeout'], capture_output=True, text=True)
        
        original_ac = result_ac.stdout.strip()
        original_battery = result_battery.stdout.strip()
        
        # 백업 파일 저장
        backup_settings = {
            'ac_timeout': original_ac,
            'battery_timeout': original_battery,
            'timestamp': datetime.now().isoformat()
        }
        
        with open('power_settings_backup.json', 'w') as f:
            json.dump(backup_settings, f, indent=2)
        
        print(f"      백업 저장: AC={original_ac}초, Battery={original_battery}초")
        
        # 절전 모드 비활성화 (0 = 비활성화)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-ac-timeout', '0'], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-battery-timeout', '0'], check=True)
        
        print("      ✅ GNOME 절전 모드 비활성화 완료")
        
        # 2. 화면 보호기 비활성화
        print("   2. 화면 보호기 비활성화...")
        
        try:
            subprocess.run(['gsettings', 'set', 'org.gnome.desktop.screensaver', 
                           'idle-activation-enabled', 'false'], check=True)
            subprocess.run(['gsettings', 'set', 'org.gnome.desktop.session', 
                           'idle-delay', '0'], check=True)
            print("      ✅ 화면 보호기 비활성화 완료")
        except:
            print("      ⚠️ 화면 보호기 설정 실패 (계속 진행)")
        
        print("   3. 안전 모드 적용 완료!")
        print("      ⚠️ Grid Search 완료 후 설정을 복원하세요!")
        
        return True
        
    except Exception as e:
        print(f"   ❌ 안전 모드 설정 실패: {str(e)}")
        print("   수동으로 절전 설정을 비활성화하거나 nohup을 사용하세요.")
        return False

def restore_power_settings():
    """백업된 전원 설정을 복원"""
    
    try:
        print("🔄 전원 설정 복원 중...")
        
        with open('power_settings_backup.json', 'r') as f:
            backup = json.load(f)
        
        # 원래 설정 복원
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-ac-timeout', backup['ac_timeout']], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.settings-daemon.plugins.power', 
                       'sleep-inactive-battery-timeout', backup['battery_timeout']], check=True)
        
        # 화면 보호기 복원
        subprocess.run(['gsettings', 'set', 'org.gnome.desktop.screensaver', 
                       'idle-activation-enabled', 'true'], check=True)
        subprocess.run(['gsettings', 'set', 'org.gnome.desktop.session', 
                       'idle-delay', '300'], check=True)
        
        print(f"   ✅ 전원 설정 복원 완료")
        print(f"   복원된 설정: AC={backup['ac_timeout']}초, Battery={backup['battery_timeout']}초")
        
        # 백업 파일 삭제
        os.remove('power_settings_backup.json')
        
        return True
        
    except FileNotFoundError:
        print("   ❓ 백업 파일이 없습니다. (이미 복원됨?)")
        return False
    except Exception as e:
        print(f"   ❌ 설정 복원 실패: {str(e)}")
        print("   수동으로 전원 설정을 복원하세요.")
        return False

# 안전 모드 적용 여부 결정
if not power_safe and APPLY_SAFE_MODE:
    print("🚨 전원 위험 감지! 안전 모드 적용 중...")
    safe_mode_applied = apply_safe_mode()
elif not power_safe:
    print("🚨 전원 위험 감지! 아래 방법 중 하나를 선택하세요:")
    print("   1. APPLY_SAFE_MODE = True로 설정 후 재실행")
    print("   2. 수동으로 절전 설정 비활성화")
    print("   3. nohup 또는 screen을 사용한 백그라운드 실행")
    safe_mode_applied = False
else:
    print("✅ 전원 설정이 안전합니다.")
    safe_mode_applied = False

# nohup 실행 방법 안내
def generate_nohup_command():
    """nohup으로 실행할 수 있는 명령어 생성"""
    
    print(f"\n🖥️ 백그라운드 실행 (nohup) 방법:")
    print(f"=" * 40)
    
    # Python 스크립트 생성
    script_content = f'''#!/usr/bin/env python3
import sys
sys.path.append('.')

# 여기에 Grid Search 실행 코드 추가
# (노트북의 주요 실행 코드를 복사해서 사용)

print("nohup으로 Grid Search 실행 중...")
# 실제 실행 코드는 노트북에서 복사하세요
'''
    
    script_filename = 'run_ultimate_grid_search.py'
    
    print(f"   1. 스크립트 파일 생성: {script_filename}")
    print(f"   2. 터미널에서 실행:")
    print(f"      cd {os.getcwd()}")
    print(f"      nohup python {script_filename} > grid_search.log 2>&1 &")
    print(f"   3. 진행 상황 확인:")
    print(f"      tail -f grid_search.log")
    print(f"   4. 백그라운드 작업 확인:")
    print(f"      jobs")
    print(f"      ps aux | grep python")
    
    print(f"\n💡 장점:")
    print(f"   • 터미널 종료해도 계속 실행")
    print(f"   • 노트북 뚜껑 닫아도 안전")
    print(f"   • 로그 파일로 진행 상황 추적")

if not power_safe or not safe_mode_applied:
    generate_nohup_command()


In [ ]:
# 수동 설정 복원 (필요시 실행)
RESTORE_SETTINGS = False  # True로 변경하면 설정 복원

if RESTORE_SETTINGS:
    print("🔄 전원 설정 수동 복원 실행...")
    success = restore_power_settings()
    
    if success:
        print("✅ 설정 복원 완료!")
    else:
        print("❌ 자동 복원 실패. 수동 복원 방법:")
        print("   1. 시스템 설정 → 전원 관리")
        print("   2. '절전 모드' 설정을 원하는 값으로 변경")
        print("   3. AC: 보통 1시간(3600초), 배터리: 15분(900초)")
        
        print("\n또는 터미널에서:")
        print("   gsettings set org.gnome.settings-daemon.plugins.power sleep-inactive-ac-timeout 3600")
        print("   gsettings set org.gnome.settings-daemon.plugins.power sleep-inactive-battery-timeout 900")
else:
    print("🔄 설정 복원이 비활성화되어 있습니다.")
    print("   복원하려면: RESTORE_SETTINGS = True로 변경 후 실행")
    
    # 현재 백업 파일 상태 확인
    if os.path.exists('power_settings_backup.json'):
        try:
            with open('power_settings_backup.json', 'r') as f:
                backup = json.load(f)
            print(f"\n📋 백업된 설정 발견:")
            print(f"   AC 절전: {backup['ac_timeout']}초")
            print(f"   배터리 절전: {backup['battery_timeout']}초")
            print(f"   백업 시간: {backup['timestamp']}")
            print(f"   복원 준비 완료!")
        except:
            print("\n❓ 백업 파일이 손상되었을 수 있습니다.")
    else:
        print(f"\n📋 백업 파일 없음 (설정이 변경되지 않았거나 이미 복원됨)")
